In [5]:
from parser_1 import read_recipe_from_url
import re
import unicodedata

In [6]:

title = u"¾ cup packed brown sugar"



In [7]:
soup = read_recipe_from_url("https://www.allrecipes.com/recipe/255365/edible-cookie-dough/")

In [16]:
unicodedata.numeric("¾")

0.75

In [29]:
units = {"teaspoon": ["t", "tsp"],
"tablespoon": ["T", "tbl", "tbs", "tbsp"],
"ounce": ["oz", "ozs", "ounce"], 
"cup":["c", "C"],
"pint": ["p", "P", "pt"], 
"quart": ["q", "Q", "qt"],
"gallon": ["g", "G", "gal"]}

In [30]:
def PluralUnit(word):
    if word[-1] == "s":
        return True, word[:-1]
    return False, word

In [32]:

ingredients = soup.find("div", {"id":"mntl-structured-ingredients_1-0"})
for child in ingredients.find_all("li"):
    text = child.text.replace("\n", "").strip()
    text = text.split(" ")

    #get QUANTITY
    quantity = unicodedata.numeric(text[0]) #quantity = chars before first space

    #get UNIT
    unit = ""
    potential_unit = text[1].replace(".","")
    #lowercase unless one char
    if len(potential_unit) > 1:
        potential_unit = potential_unit.lower()
    #handle occurence of fluid/fl before unit name
    if potential_unit == "fluid" or potential_unit == "fl":
        unit += "fluid"
        potential_unit = text[2].lower().replace(".","")
    #check for units
    for key in units.keys():
        plural, potential_unit = PluralUnit(potential_unit)
        if potential_unit == key:
            unit += potential_unit
            if plural:
                unit +="s"
        elif potential_unit in units[key]:
            unit += potential_unit
            if plural:
                unit +="s"
    if unit == "":
        unit=None
    print(text)

    #get INGREDIENT NAME
    #check for plurals



['1', 'cup', 'all-purpose', 'flour']
['¾', 'cup', 'packed', 'brown', 'sugar']
['½', 'cup', 'butter']
['1', 'teaspoon', 'vanilla', 'extract']
['½', 'teaspoon', 'salt']
['2', 'tablespoons', 'milk']
['½', 'cup', 'milk', 'chocolate', 'chips']
['½', 'cup', 'mini', 'chocolate', 'chips']
